In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7,4"
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
import numpy as np, evaluate
from sklearn.metrics import accuracy_score, f1_score
# ============ 基本配置 ============
model_name = "../models/flan-t5-base"
train_file = "/data/ykyang/datasets/pdtb3_T5/pdtb3_train_instruction.json"
dev_file   = "/data/ykyang/datasets/pdtb3_T5/pdtb3_dev_instruction.json"
test_file  = "/data/ykyang/datasets/pdtb3_T5/pdtb3_test_instruction.json"
output_dir = "../checkpoints/flan-t5-sft.json"
num_epochs = 6
batch_size = 8

# ============ 模型和分词器 ============
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ============ 数据集 ============
data_files = {"train": train_file, "validation": dev_file, "test": test_file}
raw_datasets = load_dataset("json", data_files=data_files)

# ============ 数据预处理 ============
def preprocess(examples):
    inputs = [f"{ins}\n\n{inp}" for ins, inp in zip(examples["instruction"], examples["input"])]
    targets = [t.strip() for t in examples["output"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)
    labels = tokenizer(targets, max_length=8, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = raw_datasets.map(preprocess, batched=True, remove_columns=raw_datasets["train"].column_names)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ============ 评价指标函数 ============
def compute_metrics(eval_pred):
    preds_ids, labels_ids = eval_pred
    if isinstance(preds_ids, tuple):
        preds_ids = preds_ids[0]
    preds_ids = np.where(preds_ids != -100, preds_ids, tokenizer.pad_token_id)
    labels_ids = np.where(labels_ids != -100, labels_ids, tokenizer.pad_token_id)
    
    preds = tokenizer.batch_decode(preds_ids, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1_macro": f1}

# ============ 训练参数 ============
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=500,       # 每500 step验证
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    learning_rate=5e-5,
    predict_with_generate=True,
    bf16=True,    # 使用 bf16
    fp16=False,
    logging_steps=100,
    logging_dir=f"{output_dir}/logs",
    save_total_limit=7,
    report_to="none"
)

# ============ Trainer ============
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)




Map:   0%|          | 0/1474 [00:00<?, ? examples/s]

/tmp/ipykernel_605331/3068749408.py:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [2]:
# ============ 7. 训练 ============
trainer.train()



/home/ykyang/miniconda3/envs/YYK_conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [5]:
# ============ 8. 使用最优模型在测试集上评估 ============
print("\n===== 使用验证集F1最佳模型在测试集上评估 =====")
test_results = trainer.evaluate(eval_dataset=tokenized["test"], metric_key_prefix="test")
print(test_results)


===== 使用验证集F1最佳模型在测试集上评估 =====


/home/ykyang/miniconda3/envs/YYK_conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'test_loss': 0.3726713955402374, 'test_accuracy': 0.7055630936227951, 'test_f1_macro': 0.6495265230673836, 'test_runtime': 17.4607, 'test_samples_per_second': 84.418, 'test_steps_per_second': 5.326, 'epoch': 5.0}


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,5"
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# ============ 配置 ============
model_path = "../checkpoints/flan-t5-sft.json/checkpoint-5000"  # ✅ 改成你的checkpoint路径
test_file = "/data/ykyang/datasets/pdtb3_T5/pdtb3_test_instruction.json"

max_input_length = 256
max_output_length = 8
batch_size = 8

# ============ 加载模型和分词器 ============
print(f"加载模型自: {model_path}")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.eval()

# ============ 加载测试数据 ============
dataset = load_dataset("json", data_files={"test": test_file})["test"]

def preprocess(examples):
    inputs = [f"{ins}\n\n{inp}" for ins, inp in zip(examples["instruction"], examples["input"])]
    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )
    labels = [t.strip() for t in examples["output"]]
    model_inputs["labels_text"] = labels
    return model_inputs

test_data = dataset.map(preprocess, batched=True)

# 🔧 转为 torch tensor 格式
test_data.set_format(type="torch", columns=["input_ids", "attention_mask"])

# ============ 推理 ============
loader = DataLoader(test_data, batch_size=batch_size)
preds, labels = [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("\n===== 开始测试集推理 =====")
for batch in tqdm(loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_output_length,
        )

    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    preds.extend([p.strip() for p in decoded_preds])

# ✅ 从原始 Dataset 提取 labels
labels = [l.strip() for l in test_data["labels_text"]]

# ============ 计算总体指标 ============
acc = accuracy_score(labels, preds)
f1_macro = f1_score(labels, preds, average="macro")

print("\n===== 整体评估结果 =====")
print(f"Accuracy: {acc:.4f}")
print(f"Macro-F1: {f1_macro:.4f}")

# ============ 各类别详细报告 ============
print("\n===== 分类详细报告 (PDTB3 关系类别) =====")
print(classification_report(labels, preds, digits=4))

# ============ 保存报告到文件 ============
save_path = os.path.join(model_path, "test_report.txt")
with open(save_path, "w", encoding="utf-8") as f:
    f.write("===== Overall Metrics =====\n")
    f.write(f"Accuracy: {acc:.4f}\n")
    f.write(f"Macro-F1: {f1_macro:.4f}\n\n")
    f.write("===== Per-class Report =====\n")
    f.write(classification_report(labels, preds, digits=4))
print(f"\n报告已保存到: {save_path}")


加载模型自: ../checkpoints/flan-t5-sft.json/checkpoint-5000

===== 开始测试集推理 =====


100%|██████████| 185/185 [00:12<00:00, 15.11it/s]



===== 整体评估结果 =====
Accuracy: 0.7171
Macro-F1: 0.6616

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5636    0.6039    0.5831       154
           B     0.7534    0.7278    0.7404       529
           C     0.7376    0.7869    0.7615       643
           D     0.6518    0.4932    0.5615       148

    accuracy                         0.7171      1474
   macro avg     0.6766    0.6530    0.6616      1474
weighted avg     0.7165    0.7171    0.7152      1474


报告已保存到: ../checkpoints/flan-t5-sft.json/checkpoint-5000/test_report.txt


In [9]:
import os
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# ============ 配置 ============
model_path = "../checkpoints/flan-t5-sft.json/checkpoint-5350"  # ✅ 改成你的checkpoint路径
test_file = "/data/ykyang/datasets/pdtb3_T5/pdtb3_test_instruction.json"

max_input_length = 256
max_output_length = 8
batch_size = 8

# ============ 加载模型和分词器 ============
print(f"加载模型自: {model_path}")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.eval()

# ============ 加载测试数据 ============
dataset = load_dataset("json", data_files={"test": test_file})["test"]

def preprocess(examples):
    inputs = [f"{ins}\n\n{inp}" for ins, inp in zip(examples["instruction"], examples["input"])]
    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )
    labels = [t.strip() for t in examples["output"]]
    model_inputs["labels_text"] = labels
    return model_inputs

test_data = dataset.map(preprocess, batched=True)

# 🔧 转为 torch tensor 格式
test_data.set_format(type="torch", columns=["input_ids", "attention_mask"])

# ============ 推理 ============
loader = DataLoader(test_data, batch_size=batch_size)
preds, labels = [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("\n===== 开始测试集推理 =====")
for batch in tqdm(loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_output_length,
        )

    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    preds.extend([p.strip() for p in decoded_preds])

# ✅ 从原始 Dataset 提取 labels
labels = [l.strip() for l in test_data["labels_text"]]

# ============ 计算总体指标 ============
acc = accuracy_score(labels, preds)
f1_macro = f1_score(labels, preds, average="macro")

print("\n===== 整体评估结果 =====")
print(f"Accuracy: {acc:.4f}")
print(f"Macro-F1: {f1_macro:.4f}")

# ============ 各类别详细报告 ============
print("\n===== 分类详细报告 (PDTB3 关系类别) =====")
print(classification_report(labels, preds, digits=4))

# ============ 保存报告到文件 ============
save_path = os.path.join(model_path, "test_report.txt")
with open(save_path, "w", encoding="utf-8") as f:
    f.write("===== Overall Metrics =====\n")
    f.write(f"Accuracy: {acc:.4f}\n")
    f.write(f"Macro-F1: {f1_macro:.4f}\n\n")
    f.write("===== Per-class Report =====\n")
    f.write(classification_report(labels, preds, digits=4))
print(f"\n报告已保存到: {save_path}")


加载模型自: ../checkpoints/flan-t5-sft.json/checkpoint-5350


Map:   0%|          | 0/1474 [00:00<?, ? examples/s]


===== 开始测试集推理 =====


100%|██████████| 185/185 [00:12<00:00, 15.37it/s]


===== 整体评估结果 =====
Accuracy: 0.7185
Macro-F1: 0.6666

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5575    0.6299    0.5915       154
           B     0.7629    0.7240    0.7430       529
           C     0.7368    0.7838    0.7596       643
           D     0.6579    0.5068    0.5725       148

    accuracy                         0.7185      1474
   macro avg     0.6788    0.6611    0.6666      1474
weighted avg     0.7195    0.7185    0.7173      1474


报告已保存到: ../checkpoints/flan-t5-sft.json/checkpoint-5350/test_report.txt


无数据增强的sft
===== 整体评估结果 =====
Accuracy: 0.7144
Macro-F1: 0.6592
===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5285    0.6623    0.5879       154
           B     0.7505    0.7448    0.7476       529
           C     0.7485    0.7589    0.7537       643
           D     0.6635    0.4662    0.5476       148

    accuracy                         0.7144      1474
   macro avg     0.6727    0.6581    0.6592      1474
weighted avg     0.7177    0.7144    0.7135      1474

将expansion扩展为temporal一百条+几条其他
===== 整体评估结果 =====
Accuracy: 0.6981
Macro-F1: 0.6413

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5208    0.6494    0.5780       154
           B     0.7480    0.7127    0.7299       529
           C     0.7243    0.7558    0.7397       643
           D     0.6168    0.4459    0.5176       148

    accuracy                         0.6981      1474
   macro avg     0.6525    0.6409    0.6413      1474
weighted avg     0.7008    0.6981    0.6970      1474

将expansion扩展为temporal一百条 并且删除原本的expansion
===== 整体评估结果 =====
Accuracy: 0.7069
Macro-F1: 0.6556

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5625    0.6429    0.6000       154
           B     0.7565    0.7108    0.7329       529
           C     0.7182    0.7729    0.7446       643
           D     0.6422    0.4730    0.5447       148

    accuracy                         0.7069      1474
   macro avg     0.6699    0.6499    0.6556      1474
weighted avg     0.7081    0.7069    0.7052      1474

方法二扩展expansion的86条数据为temporal
===== 整体评估结果 =====
Accuracy: 0.7096
Macro-F1: 0.6528

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.6308    0.5325    0.5775       154
           B     0.7495    0.7240    0.7365       529
           C     0.7157    0.7869    0.7496       643
           D     0.5952    0.5068    0.5474       148

    accuracy                         0.7096      1474
   macro avg     0.6728    0.6375    0.6528      1474
weighted avg     0.7069    0.7096    0.7066      1474

方法二扩展expansion的86条数据为temporal且删除原始样本
===== 整体评估结果 =====
Accuracy: 0.7049
Macro-F1: 0.6544

===== 分类详细报告 (PDTB3 关系类别) =====
              precision    recall  f1-score   support

           A     0.5808    0.6299    0.6044       154
           B     0.7578    0.6919    0.7233       529
           C     0.7103    0.7854    0.7459       643
           D     0.6283    0.4797    0.5441       148

    accuracy                         0.7049      1474
   macro avg     0.6693    0.6467    0.6544      1474
weighted avg     0.7056    0.7049    0.7028      1474

